In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import random
import string
import tensorflow as tf
import tensorflow.python.ops.rnn_cell 
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve
import collections
import matplotlib.pyplot as plt
import codecs
import time
import os
import gc
from six.moves import cPickle as pickle

from plot_module import text_plot
from plot_module import structure_vocabulary_plots
from plot_module import ComparePlots

from model_module import maybe_download
from model_module import read_data
from model_module import check_not_one_byte
from model_module import id2char
from model_module import char2id
from model_module import BatchGenerator
from model_module import characters
from model_module import batches2string
from model_module import logprob
from model_module import sample_distribution
from model_module import MODEL

In [2]:
if not os.path.exists('enwik8_filtered'):
    if not os.path.exists('enwik8'):
        filename = maybe_download('enwik8.zip', 36445475)
    full_text = read_data(filename)
    new_text = u""
    new_text_list = list()
    for i in range(len(full_text)):
        if (i+1) % 10000000 == 0:
            print("%s characters are filtered" % i)
        if ord(full_text[i]) < 256:
            new_text_list.append(full_text[i])
    text = new_text.join(new_text_list)
    del new_text_list
    del new_text
    del full_text

    (not_one_byte_counter, min_character_order_index, max_character_order_index, number_of_characters, present_characters_indices) = check_not_one_byte(text)

    print("number of not one byte characters: ", not_one_byte_counter) 
    print("min order index: ", min_character_order_index)
    print("max order index: ", max_character_order_index)
    print("total number of characters: ", number_of_characters)
    
    f = open('enwik8_filtered', 'wb')
    f.write(text.encode('utf8'))
    f.close()
    
else:
    f = open('enwik8_filtered', 'rb')
    text = f.read().decode('utf8')
    f.close() 
    (not_one_byte_counter, min_character_order_index, max_character_order_index, number_of_characters, present_characters_indices) = check_not_one_byte(text)

    print("number of not one byte characters: ", not_one_byte_counter) 
    print("min order index: ", min_character_order_index)
    print("max order index: ", max_character_order_index)
    print("total number of characters: ", number_of_characters)    

number of not one byte characters:  0
min order index:  9
max order index:  255
total number of characters:  196


In [3]:
#different
offset_1 = 0
offset_2 = 4100
valid_size_1 = 4000
valid_size_2 = 4000
valid_text_1 = text[offset_1:offset_1+valid_size_1]
valid_text_2 = text[offset_2:offset_2+valid_size_2]
train_text = text[offset_2+valid_size_2:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size_1, valid_text_1[:64])
print(valid_size_2, valid_text_2[:64])
print(valid_text_1)
print('\n\n\n')
print(valid_text_2)

99371900 , Zeno &quot;repudiated the omnipotence of the state, its interv
4000 <mediawiki xmlns="http://www.mediawiki.org/xml/export-0.3/" xmln
4000 reserve">{{Anarchism}}
'''Anarchism''' originated as a term of a
<mediawiki xmlns="http://www.mediawiki.org/xml/export-0.3/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.mediawiki.org/xml/export-0.3/ http://www.mediawiki.org/xml/export-0.3.xsd" version="0.3" xml:lang="en">
  <siteinfo>
    <sitename>Wikipedia</sitename>
    <base>http://en.wikipedia.org/wiki/Main_Page</base>
    <generator>MediaWiki 1.6alpha</generator>
    <case>first-letter</case>
      <namespaces>
      <namespace key="-2">Media</namespace>
      <namespace key="-1">Special</namespace>
      <namespace key="0" />
      <namespace key="1">Talk</namespace>
      <namespace key="2">User</namespace>
      <namespace key="3">User talk</namespace>
      <namespace key="4">Wikipedia</namespace>
      <namespace key="5">Wikipedia talk</n

In [4]:
#different
offset = 12000
valid_size = 25000
valid_text = text[offset:offset+valid_size]
train_text = text[offset+valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99343000 ves feminists, and anarcha-feminist ideas are growing.  The publ
25000 accordance with principles of equality and justice. 

Proudhon's


In [5]:
vocabulary_size = number_of_characters
vocabulary = list()
characters_positions_in_vocabulary = list()

character_position_in_vocabulary = 0
for i in range(256):
    if present_characters_indices[i]:
        vocabulary.append(unichr(i))
        characters_positions_in_vocabulary.append(character_position_in_vocabulary)
        character_position_in_vocabulary += 1
    else:
        characters_positions_in_vocabulary.append(-1)


string_vocabulary = u""
for i in range(vocabulary_size):
    string_vocabulary += vocabulary[i]
print("Vocabulary: ", string_vocabulary)
print("char2id(u'a') = %s,  char2id(u'z') = %s,  char2id(u' ') = %s" % (char2id(u'a', characters_positions_in_vocabulary),
                                                                        char2id(u'z', characters_positions_in_vocabulary),
                                                                        char2id(u' ', characters_positions_in_vocabulary)))
print("id2char(78) = %s,  id2char(156) = %s,  id2char(140) = %s" % (id2char(78,
                                                                            vocabulary),
                                                                    id2char(156,
                                                                            vocabulary),
                                                                    id2char(140,
                                                                            vocabulary)))


NameError: name 'unichr' is not defined

In [ ]:
batch_size_test=64
num_unrollings_test=10

train_batches_test = BatchGenerator(train_text,
                                    batch_size_test,
                                    vocabulary_size,
                                    characters_positions_in_vocabulary,
                                    num_unrollings_test)
valid_batches_test = BatchGenerator(valid_text_1,
                                    1,
                                    vocabulary_size,
                                    characters_positions_in_vocabulary,
                                    1)

print(batches2string(train_batches_test.next(), vocabulary))
print(batches2string(train_batches_test.next(), vocabulary))
print(batches2string(valid_batches_test.next(), vocabulary))
print(batches2string(valid_batches_test.next(), vocabulary))

In [8]:
# This class implements hierarchical LSTM described in the paper https://arxiv.org/pdf/1609.01704.pdf
# All variables names and formula indices are taken from mentioned article
# notation A^i stands for A with upper index i
# notation A_i stands for A with lower index i
# notation A^i_j stands for A with upper index i and lower index j
class HM_LSTM(MODEL):

    def not_last_ayer(self,
                      idx,                   # layer number (from 0 to self._num_layers - 1)
                      state,                 # A tuple of tensors containing h^l_{t-1}, c^l_{t-1} and z^l_{t-1}
                      bottom_up,             # A tensor h^{l-1}_t  
                      top_down,              # A tensor h^{l+1}_{t-1}
                      boundary_state_down):  # A tensor z^{l-1}_t
        # method implements operations (2) - (7) (shortly (1)) performed on idx-th layer
        # ONLY NOT FOR LAST LAYER! Last layer computations are implemented in self.last_layer method
        # and returns 3 tensors: hidden state, memory state
        # and boundary state (h^l_t, c^l_t and z^l_t accordingly)
        
        
        # batch_size of processed data
        current_batch_size = bottom_up.get_shape().as_list()[0]

    
        # note: in several next operations tf.transpose method is applied repeatedly.
        #       It was used for broadcasting activation along vectors in same batches
        
        # following operation computes a product z^l_{t-1} x h^{l+1}_{t-1} for formula (6)
        top_down_prepaired = tf.transpose(tf.transpose(state[2]) * tf.transpose(top_down))
        
        # this one cumputes a product z^{l-1}_t x h^{l-1}_t for formula (7)
        bottom_up_prepaired = tf.transpose(tf.transpose(boundary_state_down) * tf.transpose(bottom_up))
        
        # Matrix multiplications in formulas (5) - (7) and sum in argument of function f_slice
        # in formula (4) are united in one operation
        # Matrices U^l_l, U^l_{l+1} and W^l_{l-1} are concatenated into one matrix self.Matrices[idx]
        # and vectors h^l_{t-1}, z^l_{t-1} x h^{l+1}_{t-1} and  z^{l-1}_t x h^{l-1}_t are 
        # concatenated into vector X
        X = tf.concat(1, [state[0], top_down_prepaired, bottom_up_prepaired])
        concat = tf.matmul(X, self.Matrices[idx]) + self.Biases[idx]
        
        # following operations implement function vector implementation in formula (4)
        # and compute f^l_t, i^l_t, o^l_t, g^l_t and z^l_t
        [sigmoid_arg, tanh_arg, hard_sigm_arg] = tf.split(concat, [3*self.num_nodes[idx], self.num_nodes[idx], 1], axis=1)
        gate_concat = tf.sigmoid(sigmoid_arg)
        [forget_gate, input_gate, output_gate] = tf.split(gate_concat, 3, axis=1)
        modification_vector = tf.tanh(tanh_arg)
        # self.compute_boundary_state works as step function in forward pass
        # and as hard sigm in backward pass 
        boundary_state = self.compute_boundary_state(hard_sigm_arg) 
        
        # Next operations implement c^l_t vector modification and h^l_t computing according to (2) and (3)
        # Since compute_boundary_state is the step function in forward pass
        # (if argument is greater than zero z^l_t = 1, otherwise z^l_t = 0)
        # equation (2) can be implemented either using tf.cond op
        # or via summing of all options multiplied flag which value is
        # equal to 0 or 1. I preferred the second variant because it doesn't involve
        # splitting input into batches and processing them separately.
        # In this algorithm I used 3 flags: update_flag, copy_flag and flush_flag
        # update_flag = 1 if UPDATE and 0 otherwise
        # copy_flag = 1 if COPY and 0 otherwise
        # flush_flag = 1 if FLUSH and 0 otherwise
        # flags, gates and vectors are transposed for broadcasting
        update_flag = tf.transpose(tf.to_float(tf.logical_and(tf.equal(state[2], 0.), tf.equal(boundary_state_down, 1.))))
        copy_flag = tf.transpose(tf.to_float(tf.logical_and(tf.equal(state[2], 0.), tf.equal(boundary_state_down, 0.))))
        flush_flag = tf.transpose(tf.to_float(tf.logical_and(tf.equal(state[2], 1.))))
        # constant 'one' is used for building negations
        one = tf.constant([[1.]])
        tr_memory = tf.transpose(state[1])
        tr_forget_gate = tf.transpose(forget_gate)
        tr_input_gate = tf.transpose(input_gate)
        tr_output_gate = tf.transpose(output_gate)
        tr_modification_vector = tf.transpose(modification_vector)
        # new memory computation
        update_term = update_flag * (tr_forget_gate * tr_memory + tr_input_gate * tr_modification_vector)
        copy_term = copy_flag * tr_memory
        flush_term = flush_flag * tr_modification_vector
        tr_new_memory = update_term + copy_term + flush_term
        new_memory = tf.transpose(tr_new_memory)
        # new hidden states computation
        tr_hidden = tf.transpose(state[0])
        copy_term = copy_flag * tr_hidden
        else_term = (one - copy_flag) * tr_output_gate * tf.tanh(tr_new_memory)
        new_hidden = tf.transpose(copy_term + else_term)
        return new_hidden, new_memory, boundary_state
    
    def last_layer(self,
                   state,                 # A tuple of tensors containing h^L_{t-1}, c^L_{t-1} (L - total number of layers)
                   bottom_up,             # A tensor h^{L-1}_t  
                   boundary_state_down):  # A tensor z^{L-1}_t
        # method implements operations (2) - (7) (shortly (1)) performed on the last layer
        # and returns 2 tensors: hidden state, memory state (h^L_t, c^L_t accordingly)
        
        # batch_size of processed data
        current_batch_size = bottom_up.get_shape().as_list()[0]
        # last layer idx
        last = self._num_layers-1

    
        # note: in several next operations tf.transpose method is applied repeatedly.
        #       It was used for broadcasting activation along vectors in same batches
        
        # this one cumputes a product z^{l-1}_t x h^{l-1}_t for formula (7)
        bottom_up_prepaired = tf.transpose(tf.transpose(boundary_state_down) * tf.transpose(bottom_up))
        
        # Matrix multiplications in formulas (5) - (7) and sum in argument of function f_slice
        # in formula (4) are united in one operation
        # Matrices U^l_l and W^l_{l-1} are concatenated into one matrix self.Matrices[last] 
        # and vectors h^l_{t-1} and  z^{l-1}_t x h^{l-1}_t are concatenated into vector X
        X = tf.concat(1, [state[0], bottom_up_prepaired])
        concat = tf.matmul(X, self.Matrices[last]) + self.Biases[last]
        
        # following operations implement function vector implementation in formula (4)
        # and compute f^l_t, i^l_t, o^l_t, g^l_t and z^l_t
        # Note that that 'hard sigm' is omitted
        [sigmoid_arg, tanh_arg] = tf.split(concat, [3*self.num_nodes[last], self.num_nodes[last]], axis=1)
        gate_concat = tf.sigmoid(sigmoid_arg)
        [forget_gate, input_gate, output_gate] = tf.split(gate_concat, 3, axis=1)
        modification_vector = tf.tanh(tanh_arg)
        
        # Next operations implement c^l_t vector modification and h^l_t computing according to (2) and (3)
        # Check up detailed description in previous method's comments 
        # I used 2 flags: update_flag and copy_flag 
        # update_flag = 1 if UPDATE and 0 otherwise
        # copy_flag = 1 if COPY and 0 otherwise
        # flags, gates and vectors are transposed for broadcasting
        update_flag = tf.transpose(tf.to_float(tf.equal(boundary_state_down, 1.)))
        # constant 'one' is used for building negations
        one = tf.constant([[1.]])
        copy_flag = one - update_flag
        tr_memory = tf.transpose(state[1])
        tr_forget_gate = tf.transpose(forget_gate)
        tr_input_gate = tf.transpose(input_gate)
        tr_output_gate = tf.transpose(output_gate)
        tr_modification_vector = tf.transpose(modification_vector)
        # new memory computation
        update_term = update_flag * (tr_forget_gate * tr_memory + tr_input_gate * tr_modification_vector)
        copy_term = copy_flag * tr_memory
        tr_new_memory = update_term + copy_term
        new_memory = tf.transpose(tr_new_memory)
        # new hidden states computation
        tr_hidden = tf.transpose(state[0])
        copy_term = copy_flag * tr_hidden
        update_term = update_flag * tr_output_gate * tf.tanh(tr_new_memory)
        new_hidden = tf.transpose(copy_term + update_term)
        return new_hidden, new_memory
    
    @ops.RegisterGradient("HardSigmoid")
    def hard_sigm_grad(self,
                       op,                # op is operation for which gradient is computed
                       grad):             # loss partial gradients with respect to op outputs
        # This function is added for implememting straight-through estimator as described in
        # 3.3 paragraph of fundamental paper. It is used during backward pass for replacing
        # tf.sign function gradient. 'hard sigm' function derivative is 0 from minus
        # infinity to -1/a, a/2 from -1/a to 1/a and 0 from 1/a to plus infinity. Since in
        # compute_boundary_state function for implementing step function tf.sign product is
        # divided by 2, in hard_sigm_grad output gradient is equal to 'a', not to 'a/2' from
        # -1/a to 1/a in order to compensate mentioned multiplication in compute_boundary_state
        # function
        op_input = op.inputs[0]
        slope = self.slope       # parameter 'a' in 'hard sigm' function
        mask = tf.to_float(tf.logical_and(tf.greater_equal(op_input, -1./ slope), tf.less(op_input, 1. / slope)))
        return self.slope * grad * mask
        
        
    
    def compute_boundary_state(self,
                               X):
        # Elementwise calculates step function 
        # During backward pass works as hard sigm
        with self._graph.gradient_override_map({"Sign": "HardSigmoid"}):
            X = tf.sign(X)
        X = (X + tf.constant([[1.]])) / 2
        return X
    
    def iteration(self, inp, state):
        num_layers = self._num_layers
        new_state = list()
        boundaries = list()
        
        # batch_size of processed data
        current_batch_size = state[0][0].get_shape().as_list()[0]
        # activated_boundary_states variable is used as boundary_state_down
        # argument on the first layer
        activated_boundary_states = tf.constant(1., shape=[current_batch_size, 1])
        
        # The first layer is calculated outside the loop
        hidden, memory, boundary = self.not_last_layer(0,
                                                       state[-1],
                                                       inp,
                                                       state[1][0],
                                                       activated_boundary_states)
                                         
        new_state.append((hidden, memory, boundary))
        boundaries.append(boundary)
        # All layers except for the first and the last ones
        if num_layers > 2:
            for idx in range(num_layers-2):
                hidden, memory, boundary = self.not_last_layer(idx,
                                                               state[idx],
                                                               hidden,
                                                               state[idx+1][0],
                                                               boundary)
                new_state.append((hidden, memory, boundary))
                boundaries.append(boundary)
        hidden, memory = self.last_layer(state[-1],
                                         hidden,
                                         boundary)
        new_state.append((hidden, memory))
        return new_state, tf.concat(1, boundaries)
    
    def embedding_module(self,
                         train_inputs):
        current_num_unrollings = len(train_inputs)
        train_inputs = tf.concat(0, train_inputs)
        embeddings = tf.matmul(train_inputs, self.embedding_weights)
        return tf.split(embeddings, current_num_unrollings, axis=0)
        
    
    def RNN_module(self,
                   embedded_inputs,
                   saved_state):
        saved_hidden_states = list()
        for _ in range(self._num_layers):
            hidden_states.append(list())
        state = saved_state
        for emb in embedded_inputs:
            state, _ = self.iteration(emb, state)
            for layer_state, saved_hidden in zip(state, saved_hidden_states):
                saved_hidden.append(layer_state[0])
        for idx, layer_saved in enumerate(saved_hidden_states):
            saved_hidden_states[idx] = tf.concat(0, layer_state)
        return state, saved_hidden_states
            
    
    def output_module(self,
                      hidden_states):
        concat = tf.concat(1, hidden_states)
        output_module_gates = tf.transpose(tf.sigmoid(tf.matmul(concat, self.output_module_gates_weights)))
        output_module_gates = tf.split(output_module_gates, self._num_layers, axis=0)
        tr_gated_hidden_states = list()
        for output_module_gate, hidden_state in zip(output_module_gates, hidden_states):
            tr_hidden_state = tf.transpose(hidden_state)
            tr_gated_hidden_states.append(output_module_gate * tr_hidden_state)
        gated_hidden_states = tf.transpose(tf.concat(0, tr_gated_hidden_states))
        return tf.matmul(gated_hidden_states, self.output_weights) + self.output_bias
        
    
    def __init__(self,
                 batch_size,
                 vocabulary,
                 characters_positions_in_vocabulary,
                 num_unrollings,
                 num_layers,
                 num_nodes,
                 train_text,
                 valid_text,
                 embedding_size=128):
        self._results = list()
        self._batch_size = batch_size
        self._vocabulary = vocabulary
        self._vocabulary_size = len(vocabulary)
        self._characters_positions_in_vocabulary = characters_positions_in_vocabulary
        self._num_unrollings = num_unrollings
        self._num_layers = num_layers
        self._num_nodes = num_nodes
        self._train_text = train_text
        self._valid_text = valid_text
        self._valid_size = len(valid_text)
        self._embedding_size = embedding_size
        self._indices = {"batch_size": 0,
                         "num_unrollings": 1,
                         "num_layers": 2,
                         "num_nodes": 3,
                         "half_life": 4,
                         "decay": 5,
                         "num_steps": 6,
                         "averaging_number": 7,
                         "type": 8}
        self._graph = tf.Graph()
        
        self._last_num_steps = 0
        with self._graph.as_default(): 
            with self._graph.device('/gpu:0'):
                # embedding module variables
                self.embedding_weights = tf.Variable(tf.truncated_normal([self._vocabulary_size, self._embedding_size], stddev = 0.1))
                
                # RNN module variables
                self.Matrices = list()
                self.Biases = list()
                self.Matrices.append(tf.Variable(tf.truncated_normal([self._vocabulary_size + self._num_nodes[0] + self._num_nodes[1],
                                                                      4 * self._num_nodes[0] + 1],
                                                                     mean=0., stddev=0.1)))
                self.Biases.append(tf.Variable(tf.zeros([4 * self._num_nodes[0] + 1])))
                if self._num_layers > 2:
                    for i in range(self._num_layers - 2):
                        self.Matrices.append(tf.Variable(tf.truncated_normal([self._num_nodes[i] + self._num_nodes[i+1] + self._num_nodes[i+2],
                                                                              4 * self._num_nodes[i+1] + 1],
                                                                             mean=0., stddev=0.1)))
                        self.Biases.append(tf.Variable(tf.zeros([4 * self._num_nodes[i+1] + 1])))
                self.Matrices.append(tf.Variable(tf.truncated_normal([self._num_nodes[-1] + self._num_nodes[-2],
                                                                      4 * self._num_nodes[-1]],
                                                                     mean=0., stddev=0.1)))     
                self.Biases.append(tf.Variable(tf.zeros([4 * self._num_nodes[-1]])))

                dim_classifier_input = sum(self._num_nodes)
                
                # output module variables
                # output module gates weights (w^l vectors in (formula (11)))
                self.output_module_gates_weights = tf.Variable(tf.truncated_normal([dim_classifier_input, self._num_layers], stddev = 0.1))
                # classifier 
                self.output_weights = tf.Variable(tf.truncated_normal([dim_classifier_input, self._vocabulary_size], stddev = 0.1))
                self.output_bias = tf.Variable(tf.zeros([self._vocabulary_size]))
                
                """PLACEHOLDERS train data"""
                self._train_data = list()
                for _ in range(self._num_unrollings + 1):
                    self._train_data.append(
                        tf.placeholder(tf.float32, shape=[self._batch_size, self._vocabulary_size]))
                train_inputs = self._train_data[: self._num_unrollings]
                train_labels = self._train_data[1:]  # labels are inputs shifted by one time step.
                # Unrolled LSTM loop.

                saved_state = list()
                for i in range(self._num_layers-1):
                    saved_state.append((tf.Variable(tf.zeros([self._batch_size, self._num_nodes[i]]), trainable=False),
                                        tf.Variable(tf.zeros([self._batch_size, self._num_nodes[i]]), trainable=False),
                                        tf.Variable(tf.zeros([self._batch_size, 1]), trainable=False)))
                saved_state.append((tf.Variable(tf.zeros([self._batch_size, self._num_nodes[-1]]), trainable=False),
                                    tf.Variable(tf.zeros([self._batch_size, self._num_nodes[-1]]), trainable=False)))

                state = saved_state
                embedded_inputs = embedding_module(train_inputs)
                state, hidden_states = RNN_module(embedded_inputs, saved_state)
                logits = output_module(hidden_states)

                save_list = list()
                for i in range(self._num_layers-1):
                    save_list.append(saved_state[i][0].assign(state[i][0]))
                    save_list.append(saved_state[i][1].assign(state[i][1]))
                    save_list.append(saved_state[i][2].assign(state[i][2]))
                save_list.append(saved_state[-1][0].assign(state[-1][0]))
                save_list.append(saved_state[-1][1].assign(state[-1][1]))
                
                """skip operation"""
                self._skip_operation = tf.group(*save_list)

                with tf.control_dependencies(save_list):
                        # Classifier.
                    """loss"""
                    self._loss = tf.reduce_mean(
                        tf.nn.softmax_cross_entropy_with_logits(
                        logits, tf.concat(0, train_labels)))
                # Optimizer.
                self._global_step = tf.Variable(0)
                """PLACEHOLDERS half life and decay"""
                self._half_life = tf.placeholder(tf.int32)
                self._decay = tf.placeholder(tf.float32)
                """learning rate"""
                self._learning_rate = tf.train.exponential_decay(10.0,
                                                                 self._global_step,
                                                                 self._half_life,
                                                                 self._decay,
                                                                 staircase=True)
                optimizer = tf.train.GradientDescentOptimizer(self._learning_rate)
                gradients, v = zip(*optimizer.compute_gradients(self._loss))
                gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
                """optimizer"""
                self._optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=self._global_step)
                """train prediction"""
                self._train_prediction = tf.nn.softmax(logits)

                # Sampling and validation eval: batch 1, no unrolling.
                saved_sample_state = list()
                for i in range(self._num_layers-1):
                    saved_sample_state.append((tf.Variable(tf.zeros([1, self._num_nodes[i]]), trainable=False),
                                               tf.Variable(tf.zeros([1, self._num_nodes[i]]), trainable=False),
                                               tf.Variable(tf.zeros([1, self._num_nodes[i]]), trainable=False)))
                saved_sample_state.append((tf.Variable(tf.zeros([1, self._num_nodes[-1]]), trainable=False),
                                           tf.Variable(tf.zeros([1, self._num_nodes[-1]]), trainable=False))) 
                """PLACEHOLDER sample input"""
                self._sample_input = tf.placeholder(tf.float32, shape=[1, self._vocabulary_size])

                reset_list = list()
                for i in range(self._num_layers-1):
                    reset_list.append(saved_sample_state[i][0].assign(tf.zeros([1, self._num_nodes[i]])))
                    reset_list.append(saved_sample_state[i][1].assign(tf.zeros([1, self._num_nodes[i]])))
                    reset_list.append(saved_sample_state[i][2].assign(tf.zeros([1, self._num_nodes[i]])))
                reset_list.append(saved_sample_state[-1][0].assign(tf.zeros([1, self._num_nodes[-1]])))
                reset_list.append(saved_sample_state[-1][1].assign(tf.zeros([1, self._num_nodes[-1]])))
                """reset sample state"""
                self._reset_sample_state = tf.group(*reset_list)

                sample_output, sample_state, self.gates = self.iteration(self._sample_input, saved_sample_state)

                sample_save_list = list()
                for i in range(self._num_layers-1):
                    sample_save_list.append(saved_sample_state[i][0].assign(sample_state[i][0]))
                    sample_save_list.append(saved_sample_state[i][1].assign(sample_state[i][1])) 
                    sample_save_list.append(saved_sample_state[i][2].assign(sample_state[i][2])) 
                sample_save_list.append(saved_sample_state[-1][0].assign(sample_state[-1][0]))
                sample_save_list.append(saved_sample_state[-1][1].assign(sample_state[-1][1]))

                with tf.control_dependencies(sample_save_list):
                    """sample prediction"""
                    self._sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, weights, bias)) 
                
                """Matrices = list()
                Biases = list()
                gates = list()
                with tf.variable_scope(LSTM_scope, reuse=True):
                    for i in range(self._num_layers):
                        with tf.variable_scope("Cell%d" % i):
                            with tf.variable_scope("BasicLSTMCell/Linear"):
                                Matrices.append(tf.get_variable("Matrix"))
                                Biases.append(tf.get_variable("Bias"))
                cur_inp = self._sample_input
                for layer_num in range(self._num_layers):
                    concat = tf.matmul(tf.concat(1, [cur_inp,
                                                     saved_sample_state[layer_num][1]]),
                                       Matrices[layer_num]) + Biases[layer_num]
                    i, _, f, o = tf.split(1, 4, concat)
                    gates.append(tf.concat(0,
                                           [tf.sigmoid(i),
                                            tf.sigmoid(f),
                                            tf.sigmoid(o)]))
                    with tf.variable_scope(LSTM_scope, reuse=True):
                        with tf.variable_scope("Cell%d" % layer_num):
                            cur_inp, _ = cell_list[layer_num](cur_inp, saved_sample_state[layer_num])
                    
                #gates
                self.gates = tf.pack(gates)"""
                
                """saver"""
                self.saver = tf.train.Saver(max_to_keep=None)
                            
                        
    
    def _generate_metadata(self, half_life, decay, num_averaging_iterations):
        metadata = list()
        metadata.append(self._batch_size)
        metadata.append(self._num_unrollings)
        metadata.append(self._num_layers)
        metadata.append(self._num_nodes)
        metadata.append(half_life)
        metadata.append(decay)
        metadata.append(self._last_num_steps)
        metadata.append(num_averaging_iterations)
        metadata.append('reverse')
        return metadata